In [133]:
import math
import pymysql
import pandas as pd
import pprint


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from textblob import TextBlob
from IPython.display import Image
from collections import defaultdict

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

from scipy import sparse
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity

In [134]:
content = pd.read_csv('Content_Filtering.csv')

In [135]:
content.set_index('Business', inplace=True)

In [136]:
content

,Parking,Lighting,Happy Hour,Parties,Music,Ambience
Business,,,,,,
Little_Donkey,0,1,1,1,0,1
Caf__Nero,1,1,0,1,0,0
India_Pavilion,0,1,0,1,0,1
Desi_Dhaba,1,0,1,0,1,0
Thai_Sensation,1,0,1,0,1,1
Dumpling_House,0,1,0,0,1,0
Caf__Luna,0,1,1,1,0,1


In [137]:
restaurants_df = pd.read_csv('Sample_Restaurants.csv', index_col='Users')
restaurants_df.fillna(value=0, inplace=True)

In [138]:
restaurants_df.head()

,Little_Donkey,Caf__Nero,India_Pavilion,Desi_Dhaba,Thai_Sensation,Dumpling_House,Caf__Luna
Users,,,,,,,
Ajinth,4.0,0.0,5.0,0.0,3.0,0.0,2.0
Tess,3.0,3.0,3.0,0.0,4.0,0.0,0.0
Jessica,4.0,3.0,0.0,0.0,2.0,0.0,1.0
Mark,0.0,2.0,0.0,2.0,0.0,2.0,4.0
Joe,5.0,0.0,5.0,0.0,8.0,0.0,8.0


In [139]:
user = 'Ajinth'

# Helper Functions for Content Based filtering 

In [188]:
'''Function to return the list of restaurants rated by the user'''
def user_rated_list(user):
    user_rated=[]
    for key, a_res in enumerate(list(restaurants_df.loc[user, :] ==0)):
        if a_res == False:
            user_rated.append((restaurants_df.loc[user, :] ==0).index[key])
        else: 
            continue
    return user_rated  


'''Function to return the list of restaurants yet to be rated by the user'''
def user_not_rated_list(user):
    user_not_rated=[]
    for key, a_res in enumerate(list(restaurants_df.loc[user, :] ==0)):
        if a_res == True:
            user_not_rated.append((restaurants_df.loc[user, :] ==0).index[key])
        else: 
            continue
    return (user_not_rated) 


'''Function to Create the Cooccurence Matrix based on the Restaurant Attributes'''
def content_based_cooccurence_matrix_creator():
    '''Calling the user rated function'''
    user_rated = user_rated_list(user)
    rest_rest_dict = {}
    for ur in user_rated:
        user_rated_vector = content[content.index==ur].values
        rest_distance=[]
        for ar in list(restaurants_df.columns): 
            all_restaurant_vector = content[content.index==ar].values
            distance = float(pairwise_distances(user_rated_vector, all_restaurant_vector, metric='cosine'))
            similarity = 1 - distance
            rest_distance.append(distance)
        rest_rest_dict[ur] = rest_distance
    co_occurence_matrix = pd.DataFrame.from_dict(rest_rest_dict, orient='index')
    co_occurence_matrix.columns=list(restaurants_df.columns)
    return co_occurence_matrix


'''Function to create dataframe of the restaurants to be rated'''
def rest_to_rate(df):
    restaurants_to_rate = df[user_not_rated_list(user)]
    return restaurants_to_rate

In [189]:
user_rated = user_rated_list(user)
user_rated

['Little_Donkey', 'India_Pavilion', 'Thai_Sensation', 'Caf__Luna']

In [190]:
user_n_rated = user_not_rated_list(user)
user_n_rated

['Caf__Nero', 'Desi_Dhaba', 'Dumpling_House']

In [191]:
content[content.index=='Little_Donkey'].values

array([[0, 1, 1, 1, 0, 1]])

In [192]:
list(restaurants_df.columns)

['Little_Donkey',
 'Caf__Nero',
 'India_Pavilion',
 'Desi_Dhaba',
 'Thai_Sensation',
 'Dumpling_House',
 'Caf__Luna']

In [193]:
res_res_df = content_based_cooccurence_matrix_creator()

In [194]:
res_res_df

,Little_Donkey,Caf__Nero,India_Pavilion,Desi_Dhaba,Thai_Sensation,Dumpling_House,Caf__Luna
Little_Donkey,0.000000,0.422650,0.133975,0.711325,0.500000,0.646447,0.000000
India_Pavilion,0.133975,0.333333,0.000000,1.000000,0.711325,0.591752,0.133975
Thai_Sensation,0.500000,0.711325,0.711325,0.133975,0.000000,0.646447,0.500000
Caf__Luna,0.000000,0.422650,0.133975,0.711325,0.500000,0.646447,0.000000


In [195]:
restaurants_to_rate = rest_to_rate(res_res_df)
restaurants_to_rate

,Caf__Nero,Desi_Dhaba,Dumpling_House
Little_Donkey,0.422650,0.711325,0.646447
India_Pavilion,0.333333,1.000000,0.591752
Thai_Sensation,0.711325,0.133975,0.646447
Caf__Luna,0.422650,0.711325,0.646447


In [196]:
user_rated

['Little_Donkey', 'India_Pavilion', 'Thai_Sensation', 'Caf__Luna']

In [169]:
'''Bugs Fixed in the main dataset run


''' 

'''Content Based Filtering Function to Make Predictions'''

def content_based_rating_predictor():
    
    predicted_rating_dict = {}
    
    res_res_df = content_based_cooccurence_matrix_creator()
    restaurants_to_rate = rest_to_rate(res_res_df)
    
    for not_rated in restaurants_to_rate.columns: 
        column_df = pd.DataFrame(restaurants_to_rate[not_rated].sort_values(ascending=False))
        predicted_rating = []
        top_sim_list = []
        if column_df.shape[0] == 1: 
            top_sim = column_df[not_rated]            
            top_sim_list.append(top_sim)

            given_rr = column_df[not_rated].index
            
            given_rank_df = pd.DataFrame(restaurants_df.loc[user, :]) # While converting this to function the name should be an argument to the function
            given_rank = float((given_rank_df.loc[given_rr,:]).get_values())

            adjusted_predicted_rating = float(given_rank*top_sim)
            predicted_rating_dict[not_rated] = adjusted_predicted_rating
             
        else: 
            for top_n in range(2):   

                top_sim = column_df[not_rated][top_n]
                
                '''Added for bug fix'''
                if top_sim == 0: 
                    continue 
                        
                else:
                    top_sim_list.append(top_sim)

                    given_rr = column_df[not_rated].index[top_n]

                    given_rank_df = pd.DataFrame(restaurants_df.loc[user, :]) # While converting this to function the name should be an argument to the function
                    given_rank = given_rank_df.loc[given_rr,:]

                    predicted_rating_item = float(given_rank*top_sim)
                    predicted_rating.append(predicted_rating_item)
            
            if len(top_sim_list) == 1: 
                predicted_rating_dict[not_rated] = predicted_rating_item
            
            else: 

                total_sim_weight = sum(top_sim_list)
                
                if total_sim_weight == 0: 
                    predicted_rating_dict[not_rated] = 0 
                else:
                    adjusted_predicted_rating = (sum(predicted_rating))/total_sim_weight
                    predicted_rating_dict[not_rated] = adjusted_predicted_rating
        
    '''Creating the Dataframe in this step'''
    
    predicted_df = pd.DataFrame.from_dict(predicted_rating_dict, orient='index')
    predicted_df.fillna(0, inplace=True)
    predicted_df.columns=[user]
    predicted_df.sort_values([user], ascending=False, inplace=True)
    return predicted_df


In [171]:
predicted_content_model = content_based_rating_predictor()
predicted_content_model.head()

,Ajinth
Caf__Nero,3.607695
Dumpling_House,3.607695
Desi_Dhaba,2.750000
